In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
import awkward as ak
import uproot 
import mplhep as hep
import hist
import os
import time
from numba import njit
from matplotlib.colors import LogNorm
import tqdm
import pickle 
from numba import jit



from UtilityAndConst import *
from Functions import *

dir = "/Users/francescolarovere/Documents/Tesi-Triennale/Dati"
os.chdir(dir)

hep.style.use("CMS")
np.set_printoptions(threshold=np.inf)

In [3]:
files = [
    "./ntupleStubsBmtfGmt_LS133.root",
    "./ntupleStubsBmtfGmt_LS263.root",
    "./ntupleStubsBmtfGmt_LS264.root",
    "./ntupleStubsBmtfGmt_LS306.root",
]

stub_bmtf_gmt_features = [
    'orbit', 'bx', 'nStubs', 'stubHwPhi', 'stubHwPhiB', 'stubHwQual', 'stubHwEta', 'stubHwQEta', 'stubStation', 'stubWheel', 'stubSector', 'stubTag',
    'nBmtfMuons', 'bmtfHwPt', 'bmtfHwPtu', 'bmtfHwPhi', 'bmtfHwEta', 'bmtfPt', 'bmtfPtu', 'bmtfPhi', 'bmtfEta', 'bmtfHwSign', 'bmtfHwSignValid', 
    'bmtfHwDXY', 'bmtfHwQual', 'bmtfProcessor', 
    'nGmtMuons', 'gmtHwPt', 'gmtHwPtu', 'gmtHwPhi', 'gmtHwEta', 'gmtHwPhiAtVtx', 'gmtHwEtaAtVtx', 'gmtPt', 'gmtPtu', 'gmtPhi', 'gmtEta', 
    'gmtPhiAtVtx', 'gmtEtaAtVtx', 'gmtCharge', 'gmtDxy', 'gmtQual', 'gmtTfIndex']

stub_bmtf_gmt_list = []
for file in files:
    with uproot.open(file) as f:
        evs = f["bmtfNtuplizer/Events"]
        temp_awk = evs.arrays(stub_bmtf_gmt_features, library="ak")
        stub_bmtf_gmt_list.append(temp_awk)
        
bmtf_gmt = ak.concatenate(stub_bmtf_gmt_list, axis=0)



In [27]:
bx = bmtf_gmt.bx
nStubs = bmtf_gmt.nStubs
stubsStation = bmtf_gmt.stubStation
stubsSector = bmtf_gmt.stubSector
stubsWheel = bmtf_gmt.stubWheel
stubsPhiB = bmtf_gmt.stubHwPhiB
stubsHwQual = bmtf_gmt.stubHwQual

In [56]:
#Controlla che la stub successiva si trovi nella stazione dopo
def CheckStation(Station, StationP1, Direction):

   Diff = StationP1 - Station
   Index = ak.where(Diff == Direction)[0]

   return Index

#Controlla che la Wheel successiva sia +- 1 della wheel corrente, tranne per le wheel estremali
def CheckWheel(Wheel, WheelP1):
   
   IndexList = []

   if (Wheel == -2):
      Index = list(ak.where((WheelP1 == -1) | (WheelP1 == -2))[0])
      IndexList.extend(Index)

   elif (Wheel == 2):
      Index = list(ak.where((WheelP1 == 1) | (WheelP1 == 2))[0])
      IndexList.extend(Index)

   else:
      Diff = Wheel - WheelP1
      Index = list(ak.where((Diff == -1) | (Diff == 0) | (Diff == 1))[0])
      IndexList.extend(Index)

   return ak.Array(IndexList)

#Controlla che il settore successivo sia adiacente al settore corrente. Applica operazione ciclica per settori 11 e 0
def CheckSector(Sector, SectorP1):
   
   IndexList = []

   if (Sector == 11):
      Index = list(ak.where((SectorP1 == 10) | (SectorP1 == 11) | (SectorP1 == 0))[0])
      IndexList.extend(Index)

   elif (Sector == 0):
      Index = list(ak.where((SectorP1 == 11) | (SectorP1 == 0)| (SectorP1 == 1))[0])
      IndexList.extend(Index)

   else:
      Diff = Sector - SectorP1
      Index = list(ak.where((Diff == -1) | (Diff == 0) | (Diff == 1))[0])
      IndexList.extend(Index)

   return ak.Array(IndexList)


def FindSeq(arr):
    # Ordina l'array per trovare sequenze consecutive
    arr = np.sort(arr)
    sequences = []
    current_sequence = [arr[0]]

    for i in range(1, len(arr)):
        # Controlla se l'elemento corrente è consecutivo rispetto al precedente
        if arr[i] == arr[i - 1] + 1:
            current_sequence.append(arr[i])
        else:
            # Salva la sequenza trovata se ha più di un elemento
            if len(current_sequence) > 1:
                sequences.append(current_sequence)
            # Inizia una nuova sequenza con l'elemento corrente
            current_sequence = [arr[i]]

    # Aggiungi l'ultima sequenza se è valida
    if len(current_sequence) > 1:
        sequences.append(current_sequence)

    return sequences

In [30]:
Features = [nStubs, bx, stubsStation, stubsSector, stubsWheel, stubsPhiB, stubsHwQual]

In [145]:
def SearchHSCP_StationLevel(Features, Number, Forward, Backward):

   #If direction == 1 looks forward, if == -1 looks backward
   
   Muons = []

   #Feature: 0 == Stubs, 1 == bx, 2 == Station, 3 == Sector, 4 == Wheel, 5 == PhiB, 6 == Quality

   for i in tqdm.tqdm(range(Number - 1)):

      Dic = {}
      DicP1 = {}

      #Inizializzo dizionari
      for feature in ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality']:
         Dic[feature] = []
         DicP1[feature] = []

      
      if((Features[0][i] == 1 and Features[0][i+1] == 1) and Features[2][i+1] == Features[2][i] + Forward and Features[2][i] != 4):

         IndexSector = CheckSector(Features[3][i], Features[3][i+1])
         IndexWheel = CheckWheel(Features[4][i], Features[4][i+1])

         if(len(IndexSector) != 0 and len(IndexWheel) != 0):
            Match_nStubs = [1, 1]
            Dir = True

            keys = ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality']
            for idx, key in enumerate(keys, start=2):
               Dic[key].extend(Features[idx][i])
               DicP1[key].extend(Features[idx][i+1])

         else:
            continue

      elif((Features[0][i] == 1 and Features[0][i+1] == 1) and Features[2][i+1] == Features[2][i] + Backward and Features[2][i] != 1):

         IndexSector = CheckSector(Features[3][i], Features[3][i+1])
         IndexWheel = CheckWheel(Features[4][i], Features[4][i+1])

         if(len(IndexSector) != 0 and len(IndexWheel) != 0):
            Match_nStubs = [1, 1]
            Dir = False

            keys = ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality']
            for idx, key in enumerate(keys, start=2):
               Dic[key].extend(Features[idx][i])
               DicP1[key].extend(Features[idx][i+1])

         else:
            continue


      elif (Features[0][i] > 1 and Features[0][i+1] == 1):
         Sequence = FindSeq(Features[2][i])

         if len(Sequence) > 1 or len(Sequence) == 0:
            continue

         else:
            Max, Argmax = np.max(Sequence), np.argmax(Sequence)
            if Features[2][i+1] == Max + 1:
               IndexSector = CheckSector(Features[3][i][Argmax], Features[3][i+1])
               IndexWheel = CheckWheel(Features[4][i][Argmax], Features[4][i+1])

               if(len(IndexSector) != 0 and len(IndexWheel) != 0):
                  Match_nStubs = [len(Sequence[0]), 1]
                  Dir = True

                  keys = ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality']
                  for idx, key in enumerate(keys, start=2):
                     Dic[key].extend(Features[idx][i])
                     DicP1[key].extend(Features[idx][i+1])

               else:
                  continue

            else:
               continue

      elif (Features[0][i] == 1 and Features[0][i+1] > 1):

         SequenceP1 = FindSeq(Features[2][i+1])

         if len(SequenceP1) > 1 or len(SequenceP1) == 0:
            continue

         else:
            Max, Argmax = np.max(SequenceP1), np.argmax(SequenceP1)
            if Features[2][i] + 1 == SequenceP1[0][0]:
               IndexSector = CheckSector(Features[3][i+1][Argmax], Features[3][i])
               IndexWheel = CheckWheel(Features[4][i+1][Argmax], Features[4][i])

               if(len(IndexSector) != 0 and len(IndexWheel) != 0):
                  Match_nStubs = [1, len(SequenceP1[0])]
                  Dir = True

                  keys = ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality']
                  for idx, key in enumerate(keys, start=2):
                     Dic[key].extend(Features[idx][i])
                     DicP1[key].extend(Features[idx][i+1])

               else:
                  continue

            else:
               continue
              

      else:
         Sequence = FindSeq(Features[2][i])
         SequenceP1 = FindSeq(Features[2][i+1])

         if (len(SequenceP1) > 1 or len(SequenceP1) == 0): 
            continue
         elif (len(Sequence) > 1 or len(Sequence) == 0):
            continue

         else:
            Max, Argmax = np.max(Sequence), np.argmax(Sequence)

            if Max + 1 == np.min(SequenceP1):
               IndexSector = CheckSector(Features[3][i][Argmax], Features[3][i+1])
               IndexWheel = CheckWheel(Features[4][i][Argmax], Features[4][i+1])

               if(len(IndexSector) != 0 and len(IndexWheel) != 0):
                  Match_nStubs = [len(Sequence[0]), len(SequenceP1[0])]
                  Dir = True

                  keys = ['Station', 'Sector', 'Wheel', 'PhiB', 'Quality']
                  for idx, key in enumerate(keys, start=2):
                     Dic[key].extend(Features[idx][i])
                     DicP1[key].extend(Features[idx][i+1])

               else:
                  continue

            else:
               continue


      Dictionary = {
         'i': i,
         'bx': Features[1][i],
         'Matched_nStubs': Match_nStubs,
         'Dir': Dir,
         'property': Dic,
         'propertyP1': DicP1
         }


      Muons.append(Dictionary)



   return ak.Array(Muons)

In [148]:
Muons = SearchHSCP_StationLevel(Features, 10_000, 1, -1)

100%|██████████| 9999/9999 [00:09<00:00, 1048.88it/s]


In [149]:
Muons

<Array [{i: 25, bx: 489, ...}, ..., {...}] type='385 * {i: int64, bx: int64...'>